In [1]:
# Import external libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Set number of bits per cell
bpc = 2

In [3]:
# Analyze reliability, bandwidth, energy
data = []
for att in range(10, 255, 20):
    # Load and compute data
    postread = np.loadtxt(f"data/postread_{bpc}bpc_{att}.csv", delimiter=',', dtype=np.int8)
    postread_cer = np.not_equal(postread, [[i % (2**bpc) for i in range(16)]]*65536).mean()
    dt = np.loadtxt(f"data/dt_{bpc}bpc_{att}.csv", delimiter=',')
    ivdd = np.loadtxt(f"data/vdd_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivdd_dac = np.loadtxt(f"data/vdd_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivsa = np.loadtxt(f"data/vsa_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivddio = np.loadtxt(f"data/vddio_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivddio_dac = np.loadtxt(f"data/vddio_power_{bpc}bpc_{att}.csv", delimiter=',')
    power_tot = ivdd * 0.9 + ivdd_dac * 0.9 + ivsa * 0.9 + ivddio_dac * 3.3 + ivddio * 3.3
    energy_tot = power_tot * dt
    bw = 2**16 * 48 * bpc / dt
    energy_per_bit = energy_tot / 2**16 / 48 / bpc


    # Load data into confmat
    confmat = np.zeros(shape=(2**bpc, 2**bpc))
    for i in range(65536):
        for j in range(16):
            confmat[j % (2**bpc)][postread[i][j]] += 1
    confmat = confmat.astype(int)
    print(confmat)

    # Compose dataframe
    data.append({"att": att, "bpc": bpc,  "dt": dt, "power_tot": power_tot, "energy_per_bit": energy_per_bit, "bw": bw, "cer": postread_cer})
data = pd.DataFrame(data)
data

[[254005    948    931   6260]
 [ 26672 233718    407   1347]
 [ 44784  11569 202492   3299]
 [     7      6      3 262128]]
[[255274    513    441   5916]
 [  9524 247589    504   4527]
 [ 24967   9241 221831   6105]
 [     6      7      2 262129]]
[[255804    521    262   5557]
 [  8208 248618    479   4839]
 [ 16887   9249 229472   6536]
 [     7      6      4 262127]]
[[255922    667    203   5352]
 [  8267 249174    461   4242]
 [ 10573   9245 235370   6956]
 [     6      7      3 262128]]
[[256088    632    167   5257]
 [  7898 249286    449   4511]
 [ 11762   9312 235133   5937]
 [     7      7      4 262126]]
[[255999    668    204   5273]
 [  7794 249115    486   4749]
 [ 10235   9192 236710   6007]
 [     8      6      3 262127]]
[[255861    732    191   5360]
 [  7868 248670    496   5110]
 [  6417   9548 240024   6155]
 [     7      8      2 262127]]
[[255912    690    231   5311]
 [  7738 248536    510   5360]
 [  7132   9157 240102   5753]
 [     7      8      4 262125]]


,att,bpc,dt,power_tot,energy_per_bit,bw,cer
0,10,2,0.2834341526031494,0.027055,1.218823e-09,2.219724e+07,0.091775
1,30,2,0.43622374534606934,0.027173,1.884034e-09,1.442254e+07,0.058892
2,50,2,0.5472018718719482,0.026428,2.298565e-09,1.149750e+07,0.050120
3,70,2,0.6415102481842041,0.026418,2.693752e-09,9.807257e+06,0.043852
4,90,2,0.7240688800811768,0.026070,3.000379e-09,8.689030e+06,0.043815
5,110,2,0.8049533367156982,0.025904,3.314252e-09,7.815926e+06,0.042558
6,130,2,0.8763468265533447,0.026119,3.638096e-09,7.179185e+06,0.039953
7,150,2,0.9471752643585205,0.026078,3.926012e-09,6.642336e+06,0.039960
8,170,2,1.021150827407837,0.026816,4.352431e-09,6.161143e+06,0.038612
9,190,2,1.0886962413787842,0.026916,4.657569e-09,5.778890e+06,0.037816
